In [ ]:
''' Imports '''

import os
import glob

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew
from shapely import wkt
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler

from dbfread import DBF
from pyproj import CRS
import geopandas as gpd

In [ ]:
''' Paths '''

algeria_path = 'Algeria/algeria'
soil_path = 'data/soil_dz_allprops.csv'

soil_df = pd.read_csv(soil_path)
# combined_climate_seasons_df = pd.read_csv('Climate_data/Combined_Seasonal_Climate_Data.csv')
Combined_Climate_Seasonal_columns_df = pd.read_csv('data/Combined_Climate_Seasonal_columns.csv')

In [ ]:
Combined_Climate_Seasonal_columns_df

#### Load the datasets

In [ ]:
'''
    Input : - the name of the data exp : Soil, Climate, Shape...
            - in the case of the climate dataset : provide the path  

    Output : a dataframe
'''

def load_dataset (dataset_name, climat_path = None):
    if (dataset_name == "Soil"):
        return pd.read_csv(soil_path)

    elif (dataset_name == "Climat"):
        return pd.read_csv(climat_path)

    elif (dataset_name == "Shape"):
        return gpd.read_file(f'{algeria_path}.shp', encoding='utf-8')

    elif (dataset_name == "Index"):
        return gpd.read_file(f'{algeria_path}.shx', encoding='utf-8')

    elif (dataset_name == "DBF"):
        table = DBF(f'{algeria_path}.dbf', encoding='utf-8') 
        return pd.DataFrame(iter(table))

    elif (dataset_name == "PRJ"):
        with open(f'{algeria_path}.prj', 'r') as f:
            prj_text = f.read()
        return CRS.from_wkt(prj_text)

#### Missing values and Unique values

In [ ]:
''' 
    Function to calculate the amount of missing values and unique values
    Input : A dataframe
    Output : A dataframe with Three columns
            - First one : Name of the column
            - Second one : The  number of missing values
            - Third one : The number of unique values

'''
def calculate_missing_and_unique(data):
    missing_values = data.isnull().sum()
    unique_values = data.nunique()
    return pd.DataFrame({"missing_values": missing_values, "unique_values": unique_values})

In [ ]:
calculate_missing_and_unique(soil_df)

#### Central Tendencies

**skewness** : tells how much and in which direction the data deviates from a normal (symmetrical) distribution
* Positive Skewness (Right-skewed): The data's tail is longer on the right side. The mean > the median.
* Negative Skewness (Left-skewed): The data's tail is longer on the left side. The mean < the median.
* Skewness of 0: The data is perfectly symmetrical 

In [ ]:
''' 
    Function that calculates the central tendencies of an attribute.
        Input : A dataframe and the name of the column
        Ouput : A dict with the value of :
                - The mean
                - The median
                - The mode
                - The symmetry
                - The skewness 
'''
def calculate_central_tendency(data, column):
    mean = data[column].mean()
    median = data[column].median()
    mode = data[column].mode()

    if len(mode) > 1:  # Check if there are multiple modes
        symmetry = "Skewed (multimodal)" 
    else:
        mode_value = mode.iloc[0]  # Use the first mode value
        symmetry = "Symmetric" if np.isclose(mean, median) and np.isclose(mean, mode_value) else "Skewed" #(skewed == asymétrique)

    skewness = skew(data[column].dropna())  # Drop NaNs for skewness calculation 
    

    return {
        "mean": mean,
        "median": median,
        "mode": mode.to_list(),
        "symmetry": symmetry,
        "skewness": skewness
    }

In [ ]:
calculate_central_tendency(Combined_Climate_Seasonal_columns_df, 'Summer_PSurf_max')

# calculate_central_tendency(soil_df, 'clay % subsoil')

In [ ]:
''' 
    To display the central tendencies of all the columns of the soil dataset and ignore the geometry column
'''
for column in soil_df.columns:
    try:
        print(column, " : ", calculate_central_tendency(soil_df, column))
    except TypeError as e:
        continue


#### Measures of dispersion + outliers

* Standard deviation (l’écart-type) :
    * Quantifies how much the values deviate from the mean of the dataset.
    * A low standard deviation : means that the values are close to the mean, indicating low variability in the data.
    * A high standard deviation : means that the values are spread out over a wider range, indicating high variability.

* Variance : 
    * Gives a measure of the spread of data points in squared units (if the data is in meters, the variance will be in square meters).
    * Tells how far each data point is from the mean, on average.

* Both **std_dev** and **variance** are measures of the variability or spread of a dataset, but **std_dev** is more commonly used because it is in the original units of the data, making it easier to interpret.


In [ ]:
''' 
    Function to calculat the dispersion of the data
    Input : A dataframe + The name of the attribute
    Output : A dict with :
            - The standard deviation
            - The variance
            - The min value
            - The median
            - The max value
            - The IQR 
            - A list of outliers (Show the full row and its index in the dataframe)
'''
def calculate_dispersion(data, column):

    std_dev = data[column].std() 
    variance = data[column].var()
    
    Q0 = data[column].quantile(0)
    Q1 = data[column].quantile(0.25)
    Q2 = data[column].quantile(0.50)
    Q3 = data[column].quantile(0.75)
    Q4 = data[column].quantile(1)

    iqr = Q3 - Q1
    lower_bound = Q1 - 1.5 * iqr
    upper_bound = Q3 + 1.5 * iqr
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]

    return {
        "std_dev": std_dev,
        "variance": variance,
        "min": Q0,
        "median": Q2,
        "max": Q4,
        "iqr": iqr,
        "outliers": outliers
    }

In [ ]:
dispersion = calculate_dispersion(Combined_Climate_Seasonal_columns_df, 'Fall_PSurf_max')

In [ ]:
dispersion['outliers']

#### Plots

##### BoxPlot with Outliers

In [ ]:
''' 
    Function to construct boxplots and display outliers
    Input : - A dataframe
            - The attribute name
            - A bool value to Show outliers or not
    Output : A boxplot      
 '''
def plot_boxplot(data, columns, show_outliers=True):
    
    plt.boxplot([data[column] for column in columns],
                vert=True, #Orientation of the box plot = vertical (False is horizental)
                patch_artist=True, # Enables filling the box with color.
                showfliers=show_outliers,
                boxprops=dict(facecolor='lightblue', color='black'), 
                medianprops=dict(color='red'), 
                whiskerprops=dict(color='black'),
                capprops=dict(color='orange'))
    
    plt.title("Box Plots")
    plt.ylabel("Values")
    plt.xticks(range(1, len(columns) + 1), columns, rotation=45, ha='right')  # Set x-ticks to column names
    
    plt.show()

In [ ]:
plot_boxplot(soil_df, ['sand % topsoil', 'sand % subsoil', 'silt % topsoil', 'silt% subsoil', 'clay % subsoil'])

##### Histograms and data distribution

In [ ]:
''' 
    Function to construct histograms and visualize the data distribution
    Input : A dataframe + The attribute name
    Ouput : A histogramme plot     
'''

def plot_histogram(data, column):
    
    sns.histplot(data[column].dropna(), kde=True)
    plt.title(f"Histogram of {column}")
    plt.xlabel(column)
    plt.ylabel("Frequency")
    plt.show()


In [ ]:
for column in soil_df.columns:
    if column == 'geometry':
        continue  # Skip this column
    plot_histogram(soil_df, column)


In [ ]:
plot_histogram(df, 'sand % topsoil')

##### Scatter plots and correlations

In [ ]:
''' 
    Function to construct scatter plots
    Input : - A dataframe
            - The first attribute name
            - The second attribute name
            
    Ouput : - A scatter plot
            - The % of correlation
'''
def plot_scatter(data, column1, column2):

    sns.scatterplot(data=data, x=column1, y=column2)

    plt.title(f"Scatter plot of {column1} vs {column2}")
    plt.xlabel(column1)
    plt.ylabel(column2)
    
    correlation = data[[column1, column2]].corr().iloc[0, 1]
    print(f"Correlation between {column1} and {column2}: {correlation}")
    plt.show()

In [ ]:
plot_scatter(soil_df, 'sand % topsoil', 'sand % subsoil')

In [ ]:
plot_scatter(df, 'silt % topsoil', 'silt% subsoil')

##### Map Plot

In [ ]:
''' 
    Funtion to plot the soil map 
    Input : A dataframe + The name of the attribute
    Output : A map plot 
'''
def plot_soil_map(soil_df, attribute):

    # Convertir en GeoDataFrame
    gdf = gpd.GeoDataFrame(soil_df, geometry='geometry', crs="EPSG:4326")

    # Tracer la carte avec le colormap "viridis" pour correspondre aux couleurs de l'image
    fig, ax = plt.subplots(figsize=(5, 5))
    # cmap='YlOrBr'
    gdf.plot(column=attribute, cmap='viridis', legend=True, 
            legend_kwds={'label': attribute, 'orientation': "horizontal"},
            ax=ax)

    # Personnaliser le graphique
    plt.title(f"{attribute} across Algeria")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.show()

plot_soil_map(soil_df, 'sand % topsoil')

In [ ]:
''' 
    Funtion to plot the climate map 
    Input : A dataframe + The name of the attribute
    Output : A map plot 
'''
def plot_climate_map(Combined_Climate_Seasonal_columns_df, attribute):

    df_data = Combined_Climate_Seasonal_columns_df[['lon', 'lat', attribute]]
    pivoted_data = df_data.pivot_table(index="lat", columns="lon", values=attribute, aggfunc="mean")

    # Define the geographic extent of the heatmap (min and max longitude and latitude)
    extent = [df_data['lon'].min(), df_data['lon'].max(), df_data['lat'].min(), df_data['lat'].max()]

    # Plot the heatmap using imshow with the defined extent
    plt.figure()
    ax = plt.gca()
    im = ax.imshow(pivoted_data, cmap="coolwarm", extent=extent, origin="lower", aspect="auto")
    plt.colorbar(im, label=attribute)

    gdf_algerie = gpd.read_file(f'{algeria_path}.shp', encoding="utf-8")
    gdf_algerie.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)

    # Add titles and labels
    plt.title(f"{attribute} Heatmap")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    plt.show()

plot_climate_map(Combined_Climate_Seasonal_columns_df, 'Spring_PSurf_max')

In [ ]:
''' 
    Funtion to plot a country's map
    Input : The path to the shape file
    Output : A map plot 
'''
def plot_algeria_map(path):
    # Load the shapefile
    gdf_shp = gpd.read_file(f'{path}', encoding='utf-8')
    country_name = path.split("/")[0].split(".")[0]
    # Plot
    fig, ax = plt.subplots()
    gdf_shp.plot(ax=ax, color="lightgreen", edgecolor="black")

    # Add title and labels
    ax.set_title(f"Map of {country_name}")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    plt.show()

plot_algeria_map(f'{algeria_path}.shp')

#### Data Reduction through Aggregation by Seasons:

In [ ]:
''' 
    Function to get the data's season based on its time 
    Input : A dataframe' time column
    Output : The season's name (Fall, Winter, Spring, Summer)
'''
def get_season(date):
    if date >= pd.Timestamp(year=date.year, month=12, day=22) or date < pd.Timestamp(year=date.year, month=3, day=22):
        return 'Winter'
    elif date >= pd.Timestamp(year=date.year, month=3, day=22) and date < pd.Timestamp(year=date.year, month=6, day=22):
        return 'Spring'
    elif date >= pd.Timestamp(year=date.year, month=6, day=22) and date < pd.Timestamp(year=date.year, month=9, day=22):
        return 'Summer'
    else:
        return 'Fall'

In [ ]:
''' 
    Function to aggregate data by season
    Input : - The file pattern (In this case we have 12 files (One for each month) for each type of climate )
            - The climate attribute name (Exp : PSurf or Qair or Wind ...)

    Output : A dataframe of that specific climate attribute with :
            - Dropped time column
            - all the data grouped by longitude - latitude - season
            - Only the (Min - Max - Mean) values
'''
def seasonal_aggregation(file_pattern, variable_name):
    # Load all monthly files into a single DataFrame
    all_months = pd.concat([pd.read_csv(f) for f in glob.glob(file_pattern)], ignore_index=True)
    
    # Convert 'time' to datetime (originally 'object' converted to 'datetime64[ns]')
    all_months['time'] = pd.to_datetime(all_months['time'])
    
    # Assign season name based on date
    all_months['season'] = all_months['time'].apply(get_season)
    
    # Group by latitude, longitude, season, and calculate min, max, mean
    seasonal_data = all_months.groupby(['lat', 'lon', 'season']).agg(
        min_value=(variable_name, 'min'),
        max_value=(variable_name, 'max'),
        mean_value=(variable_name, 'mean')
    ).reset_index()
    
    return seasonal_data

In [ ]:
# Paths to CSV files for each dataset (files patterns)
dataset_info = {
    'PSurf': 'Climate_data/csv_filtered_climat_data/PSurf_WFDE5_CRU_2019*_v2.csv',
    'Qair': 'Climate_data/csv_filtered_climat_data/Qair_WFDE5_CRU_2019*_v2.csv',
    'Rainf': 'Climate_data/csv_filtered_climat_data/Rainf_WFDE5_CRU_2019*_v2.csv',
    'Snowf': 'Climate_data/csv_filtered_climat_data/Snowf_WFDE5_CRU_2019*_v2.csv',
    'Tair': 'Climate_data/csv_filtered_climat_data/Tair_WFDE5_CRU_2019*_v2.csv',
    'Wind': 'Climate_data/csv_filtered_climat_data/Wind_WFDE5_CRU_2019*_v2.csv'
}

In [ ]:
# Perform aggregation for each dataset and save it as a csv file
seasonal_aggregates = {}
for variable, path in dataset_info.items():
    seasonal_aggregates[variable] = seasonal_aggregation(path, variable)
    # seasonal_aggregates[variable].to_csv(f'Climate_data/Seasonal_filtered_climate_data/{variable}.csv', index=False)


#### Data Integration: Merges Data from Multiple Sources into a Single Coherent Dataset:

##### Merge Seasonal Climate datasets

Just to :
* Put all the seasons in one dataset, keeping only the (min - max - mean) values 
* turn the seasons into columns 

In [ ]:
'''
    Function to merge the seasonal datasets into one single dataset, and keeping only min - max - mean values

    input : path to the files = Climate_data/Seasonal_filtered_climate_data/*.csv
    output : saves the dataset into a csv file
'''
def merge_climate_datasets():
    # Path to the saved seasonal CSV files
    file_paths = glob.glob('Climate_data/Seasonal_filtered_climate_data/*.csv')

    # Initialize an empty list to store each dataset
    dataframes = []

    # Read each file and add it to the list of dataframes
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        variable_name = file_path.split("\\")[-1].split('.')[0] # Extract variable name from filename
        df = df.rename(columns={'min_value': f'{variable_name}_min',
                                'max_value': f'{variable_name}_max',
                                'mean_value': f'{variable_name}_mean'})
        dataframes.append(df)

    # Merge all dataframes on common columns
    merged_df = dataframes[0]
    for df in dataframes[1:]:
        merged_df = pd.merge(merged_df, df, on=['lat', 'lon', 'season'], how='outer')

    # Save the merged dataset to a new CSV file
    # merged_df.to_csv('Climate_data/Combined_Seasonal_Climate_Data.csv', index=False)

# merge_climate_datasets()

In [ ]:
'''
    Function to pivot the dataset to make seasons as columns and save the new dataset into a csv file
'''
def pivot_climate_seasons():
    combined_seasoned_climat_df = pd.read_csv('Climate_data/Combined_Seasonal_Climate_Data.csv')
    pivoted_df = combined_seasoned_climat_df.pivot_table(
        index=['lat', 'lon'],
        columns='season',
        values=[col for col in combined_seasoned_climat_df.columns if col not in ['lat', 'lon', 'season']]
    )

    # Flatten the multi-index columns
    pivoted_df.columns = [f"{season}_{metric}" for metric, season in pivoted_df.columns]

    # Reset the index if needed
    pivoted_df = pivoted_df.reset_index()

    # Save the transformed dataframe
    pivoted_df.to_csv('Combined_Climate_Seasonal_columns.csv', index=False)

# pivot_climate_seasons()

##### Merge Climate dataset with soil dataset 

In [ ]:
'''
    Function to Merge the climate dataset with the soil dataset based on spatial relationships.
    
    Input : The climate datafram + The soil dataframe
    Output : One single dataframe with concatenated data
'''
def merge_climate_soil(climate_df, soil_gdf):
    # Ensure the soil dataset is a GeoDataFrame
    soil_gdf['geometry'] = soil_gdf['geometry'].apply(wkt.loads)
    soil_gdf = gpd.GeoDataFrame(soil_df, geometry='geometry', crs="EPSG:4326")
    
    # Create GeoDataFrame for the climate dataset
    climate_gdf = gpd.GeoDataFrame(
        climate_df,
        geometry=[Point(lon, lat) for lon, lat in zip(climate_df['lon'], climate_df['lat'])],
        crs="EPSG:4326"  # Assuming WGS84 coordinate system
    )
    
    # Perform spatial join
    merged_gdf = gpd.sjoin(climate_gdf, soil_gdf, how='inner', predicate='within')
    
    return merged_gdf


In [ ]:
merged_climate_soil_df = merge_climate_soil(Combined_Climate_Seasonal_columns_df, soil_df)

In [ ]:
merged_climate_soil_df 

#### Multiple Choices of Handling Outliers and Missing Values:

In [ ]:
''' 
    Function to either delete the missing data or replace it by the mean or the median
    input : - A dataframe
            - The attribute name
            - The method (remove / mean / median)

    Output : Dataframe without missing values  
'''

def handle_missing_data(data, column, method='remove'):
    # Handle missing values
    if method == 'remove':
        data = data.dropna(subset=[column])
    elif method == 'mean':
        data[column] = data[column].fillna(data[column].mean())
    elif method == 'median':
        data[column] = data[column].fillna(data[column].median())

    return data


In [ ]:
''' 
    Function to handle outliers
    input : - A dataframe
            - The attribute name
            - The method (IQR)

    Output : Dataframe without outliers
'''

def handle_outliers(data, column, outlier_method='IQR'):

    # Handle outliers (using IQR method)
    if outlier_method == 'IQR':
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        iqr = Q3 - Q1
        lower_bound = Q1 - 1.5 * iqr
        upper_bound = Q3 + 1.5 * iqr
        data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

    return data


In [ ]:
# Test
combined_seasoned_climat_df = pd.read_csv('Combined_Climate_Seasonal_columns.csv')
for column in combined_seasoned_climat_df.columns:
    if column in ['lat', 'lon', 'season']:
        continue  # Skip this column
    combined_seasoned_climat_df = handle_missing_data(combined_seasoned_climat_df, column)
    combined_seasoned_climat_df = handle_outliers(combined_seasoned_climat_df, column, outlier_method='IQR')

In [ ]:
plot_boxplot(combined_seasoned_climat_df, ['Fall_PSurf_min', 'Fall_PSurf_max', 'Fall_PSurf_mean'])

#### Data Normalization: Min-Max / Z-Score Methods:

In [ ]:
''' 
    Function to normalize the data with one of 2 methodes (Z-score or Min-Max)
    Input : - A Dataframe
            - The attribute name
            - the normalization method name
    Output : The dataframe with normalized data
'''

def normalize_data(data, column, method='z-score'):
    if method == 'z-score':
        if (data[column].std()) == 0: 
            data[column] = 0
        else: 
            data[column] = (data[column] - data[column].mean()) / data[column].std()
    elif method == 'min-max':
        scaler = MinMaxScaler()
        data[column] = scaler.fit_transform(data[[column]])
        
    return data


In [ ]:
# Test for all the columns of the climate dataset
Combined_Climate_Seasonal_columns_df = pd.read_csv('Combined_Climate_Seasonal_columns.csv')
for column in Combined_Climate_Seasonal_columns_df.columns:
    if column in ['lat', 'lon', 'season']:
        continue  # Skip this column
    d = normalize_data(Combined_Climate_Seasonal_columns_df, column, method='min-max')

d

#### Data Reduction via Discretization of Continuous Data: Equal Frequency / Amplitude:

In [ ]:
''' 
    Function to discretize data with one of 2 methodes (equal-amplitude or equal-frequency)
     Input : - A Dataframe
            - The attribute name
            - the discretization method name
    Output : The dataframe with discretized data
'''
def discretize_data(data, column, method='equal-frequency'):
    n = len(data[column])
    K = int(1 + (10 / 3) * np.log10(n))
    print(f"The number of intervals = {K}\n")

    if method == 'equal-amplitude':
        # Equal-amplitude (equal-width) binning
        min_value = data[column].min()
        max_value = data[column].max()
        width = (max_value - min_value) / K
        bins = [min_value + i * width for i in range(K + 1)]
        print(f"The interval's bins (equal-amplitude) are: {bins}\n")

        # Calculate labels as mean values for each bin
        labels = []
        for i in range(K):
            bin_data = data[(data[column] >= bins[i]) & (data[column] < bins[i + 1])]
            mean_value = bin_data[column].mean() if not bin_data.empty else (bins[i] + bins[i + 1]) / 2
            labels.append(mean_value)
        print(f"The interval's labels (equal-amplitude) are: {labels}\n")

        d = pd.cut(data[column], bins=bins, labels=labels)
        data[column] = d
        
    elif method == 'equal-frequency':
        # Equal-frequency binning
        d = pd.qcut(data[column], q=K, labels=False, duplicates='drop')
        
        # Calculate mean for each equal-frequency bin
        labels = []
        for bin_number in range(K):
            bin_data = data[d == bin_number]
            mean_value = bin_data[column].mean()
            labels.append(mean_value)
        print(f"The interval's labels (equal-frequency) are: {labels}\n")
        
        # Reassign the discretized column with mean labels
        d = d.map(lambda x: labels[int(x)] if pd.notna(x) else None)

    data[column] = d

    return data


In [ ]:
# test on the climate dataset
Combined_Climate_Seasonal_columns_df = pd.read_csv('Combined_Climate_Seasonal_columns.csv')
d = discretize_data(Combined_Climate_Seasonal_columns_df, 'Fall_PSurf_max', method='equal-amplitude')

In [ ]:
d

#### Data Reduction (Elimination of Redundancies) Horizontal / Vertical:

In [ ]:
''' 
    Function to eliminate redundancies horizontaly or vertically
    Input : - A dataframe
            - The reduction methode
            - The columns names in the case of the vertical reduction

    Output : Dataframe without redundancies
'''
def eliminate_redundancies(data, method='vertical', columns=None):
    if method == 'horizontal':
        data = data.drop_duplicates()
    elif method == 'vertical' and columns:
        data = data[columns]  # Keep only the necessary columns
    return data
